### VERSION 2.0

In [1]:
import numpy as np
import random
import math
import os
import time
import re
import string
from tqdm.notebook import tqdm_notebook

from itertools import permutations,combinations_with_replacement,combinations

In [2]:
#get a list of all lowercase letters of the alphabet to check later
letters = list(string.ascii_lowercase)
len(letters)

26

In [3]:
#a list of all 12972 possible guesses for the game
guesses = open('/content/Worlde_Word_List.txt','r').read().splitlines()
print(len(guesses))
#print(guesses[:10])
TOTAL_WORDS=len(guesses)

12972


In [4]:
#a list of all 2315 possible naswer to the game
answers = open('/content/Wordle_Possible_Answers.txt','r').read().splitlines()
print(len(answers))
#print(guesses[:10])
TOTAL_ANS=len(answers)

2315


In [5]:
#get a dictionary with all words along with their probability of occurences

long_freqs = open('/content/long_freqs.txt','r').read().splitlines() #12972 words
short_freqs = open('/content/short_freqs.txt','r').read().splitlines() #2315 words 

print(len(long_freqs),len(short_freqs))
#print(len(long_freqs)+len(short_freqs))
freqs = long_freqs + short_freqs
#print(len(freqs))

freqs_dict={}
for item in freqs:
  x = item.split(' ')
  freqs_dict[x[0]] = float(x[1])
print(len(freqs_dict))
del long_freqs, short_freqs, #freqs

10657 2315
12972


In [6]:
#Change -1 to 1e-16,so that math domanin error doesn't show up
counter =0
for item in freqs_dict.items():
  itm = list(item)
  if itm[-1] == -1.:
    freqs_dict[itm[0]] = 1e-16
    counter +=1
counter

70

**This function eliminates words based on the word we gussed that is the occurance of the letters in grey, yellow and the greens**

In [7]:
def eliminate_words(not_letters, present_letters,pos_letters,pos_absent):
  letter_counter = 0
  remain = []

  for item in pos_letters:
    if item in letters:#letters = all 26 letters
      letter_counter +=1
  #print('letter_counter:',letter_counter)
  #letter_counter keeps counts of how many letters are in the fixed position

      
  #print(not_letters)
  for item in guesses:
    temp = list(item)
    #print(temp)
    #eliminates letters that aren't present
    if ((len((set(temp) & set(not_letters)))) == 0) and\
                                      ((len((set(temp) & set(present_letters)))) == len(present_letters)): #keeps letters are
      if letter_counter!=0:
        counter = letter_counter
        five_counter = 0
        while five_counter <5:
          current_letter = pos_letters[five_counter]
          if current_letter in letters:
            #print('here2',current_letter,temp[five_counter])

            if temp[five_counter] == current_letter:
              #print('here3',item)
              counter -=1
              five_counter +=1
              #print(counter,five_counter)
              if (counter == 0) or (five_counter == 5) :
                remain.append(item)
            else:
              break #since if one letter doesn't match, it doesn't matter if others match
          else:
            five_counter+=1
            
      else:#letter_counter !=0
        remain.append(item)

  #eliminate letter based on yellow positions
  new_remain=[]
  do_eliminate=False
  for x in pos_absent:
    if len(list(x))!=0:
      do_eliminate=True
      break

  
  if do_eliminate:
    
    for word in remain:
      match=False
      list_word = list(word)
      for indx, pos in enumerate(pos_absent):
        if match:
          break
        pos_list = list(pos)
        if len(pos_list) !=0:
          #county=len(pos_list)
          for letter in pos_list:
            if list_word[indx]==letter:
              match=True
              break
        if not match and indx==4:
          new_remain.append(word)
    return new_remain
  
  else:
    return remain

**This function calculates the information gain of each word and then sorts a dictionary with the highest information gained word at the top**

In [8]:
def get_entropy2(remain,freqs,combs):
  bit_info={}
  bit_relv={}

  prob_average=[]
  counter = 0
  for item in remain:
    #if counter%100==0:
      #print('CCC:',counter)
    counter +=1
    temp = list(item)
    #eliminates letters that aren't present
    probs_list=[]
    rel_probs=[]
    EI_list=[]
    EI_relv=[]
    for comb in list(combs):
      
      not_letters = []
      present_letters = []
      pos_letters = []
      letter_counter=0
      remain_words=[]
      for indx,comb_item in enumerate(comb):
        #counterss +=1
        #print(indx,comb_item)
        if comb_item == 'green':
          pos_letters.append(temp[indx])
        elif comb_item == 'yellow':
          present_letters.append(temp[indx])
          pos_letters.append('-')
        else: #comb_item == grey
          not_letters.append(temp[indx])
          pos_letters.append('-')
      #print('HERE:',word,pos_letters,present_letters,not_letters)
      for ltr in pos_letters:
        if ltr in letters:#letters = all 26 letters
          letter_counter +=1
      #print('le',letter_counter)
      
      for word in remain:                        
        
        temp1 = list(word)
        if ((len((set(temp1) & set(not_letters)))) == 0) and\
                                          ((len((set(temp1) & set(present_letters)))) == len(present_letters)): #keeps letters are
          if letter_counter!=0:
            counter = letter_counter
            five_counter = 0
            while five_counter <5:
              current_letter = pos_letters[five_counter]
              if current_letter in letters:
                #print('here2',current_letter,temp[five_counter])

                if temp1[five_counter] == current_letter:
                  #print('here3',item,word)
                  counter -=1
                  five_counter +=1
                  #print(counter,five_counter)
                  if (counter == 0) or (five_counter == 5) :
                    remain_words.append(word)
                else:
                  #pass
                  break #since if one letter doesn't match, it doesn't matter if others match
              else:
                five_counter+=1
                
          else:#letter_counter !=0
            
            remain_words.append(word)
      
      if len(remain_words)!=0:
        relv_sums =0
        for wordss in remain_words:
          relv_prob = freqs[wordss]
          #if relv_prob !=0 and relv_prob>0:
          #relvs = math.log2(1/relv_prob)
          relv_sums += relv_prob* math.log2(1/relv_prob)


        probs = len(remain_words)/len(remain)
        log2 = math.log2(1/probs)
        probs_list.append(probs)
        EI_list.append(probs*log2)
        EI_relv.append(relv_sums)

    bit_info[item]=sum(EI_list)
    bit_relv[item] = sum(EI_relv)
  #print('BIT:',len(bit_relv))
  
    
  
  norm_sum = np.sum(np.asarray(list(bit_relv.values())))
  norm_freqs = {key:bit_relv[key]/norm_sum for key in bit_relv.keys()}
    
  sorted_entropy = sorted(bit_info.items(), key=lambda kv: kv[1],reverse=True)
  sorted_entropy_relv = sorted(norm_freqs.items(), key=lambda kv: kv[1],reverse=True)
  return sorted_entropy,sorted_entropy_relv

**This uses permutations and combinations to capture all the different possible ways in which green,grey and yellow letters can be arranged for a word**

In [9]:
combx = combinations_with_replacement(['grey', 'yellow', 'green'], 3)
#print(len(list(combx)))

PERMS=[]
for item in list(combx):
  x = list(permutations(item, 3))
  #for y in x:
    #if y not in PERMS:
  PERMS.extend(x)
#print(len(PERMS))
temp=[]
COMBS=[]
for permu in PERMS:
  combs = list(combinations_with_replacement(permu, 5))
  temp.extend(combs)
for item in temp:
  if item not in COMBS:
    COMBS.append(item)
print('TOTAL COMBINATIONS:',len(COMBS))

TOTAL COMBINATIONS: 99


**This Function plays the game, taking the word guessed comparing it with the answer and generating the grey,yellow and green combinations**

In [10]:
def play(answer, guess_word,abs,pres,pos_pres, pos_abs):

  ans =list(answer)
  guess = list(guess_word)
  
  #temp_abs = []
  #temp_pres = []

  temp_abs = list(set(guess).difference(set(ans))) #grey
  temp_pres = list(set(guess).intersection(set(ans))) #yellow

  #print(temp_abs,' ', temp_pres)

  abs.extend(list(set(temp_abs)))
  pres.extend(list(set(temp_pres)))

  abs = list(set(abs))
  pres = list(set(pres))

  #print(abs,' ', pres)


  for indx,guess_letter in enumerate(guess):
    ans_letter = ans[indx]
    #print(indx,guess_letter,ans_letter)
    if ans_letter == guess_letter:
      pos_pres[indx] = guess_letter #green
      #pos_abs.append('')
      if guess_letter in pres:
        pres.remove(guess_letter)
    else:
      #pos_pres.append('-')
      #if guess_letter not in pres:
      if guess_letter not in abs and guess_letter in pres:
        #print(indx,pos_abs[indx])
        if pos_abs[indx]=='':
          pos_abs[indx] = guess_letter
        else:
          temp = pos_abs[indx]
          #print('temp',type(temp))
          #print(type(guess_letter))
          pos_abs[indx] = temp + guess_letter



  return abs,pres,pos_pres,pos_abs

Plays one game of Wordle, either by custom choosing the answer and the first guess, or randomly choosing the first guess or randomly choosing both the answer and the first guess

In [18]:

#ANSWER
answer = random.choice(answers) #random word from the list of answers
#answer='wonky' #or one can chose any word

#FIRST_GUESS
guess = random.choice(guesses) #random first guess from the list of guesses(12,297 words)
#first_guess= 'slate' #or one can choose any word


print('ANSWER:',answer)
absent,present,pos_present,pos_absent = play(answer, guess,[],[],list('-----'),['','','','',''])
print('1 guess:',guess)

remain_words = []

for number_play in list(range(2,7)):
  if number_play < 7:
      
    if list(answer) != list(guess):
      remain_words = eliminate_words(absent, present, pos_present,pos_absent)
      sorted_entropyx,sorted_relv_entropyx = get_entropy2(remain_words,freqs_dict,COMBS)
      guess = sorted_entropyx[0][0]
      print(number_play,'guess:',guess)
      #play
      absent,present,pos_present,pos_absent = play(answer, guess,absent,present,pos_present,pos_absent)
      
      
    if list(answer)==list(guess):
      print('{} guessed in {} guesses'.format(guess,number_play))
      break
      

  if number_play==6:

    if list(answer) == list(guess):
      print('Word Guessed Right at the Last step:',guess)

    else:
      print('LOST')
      print('End of the Game & the Final Guess:',guess)

ANSWER: blink
1 guess: fains
2 guess: bring
3 guess: blink
blink guessed in 3 guesses


## Plays Wordle on its own, by picking a random answer and then making a random first guess and thereafter playing based on the word that gives the highest information

In [19]:
NUM_GAMES = 5 #Number of games to play
NUM_WINS = 0
win_dict = {'1':0,'2':0,'3':0,'4':0,'5':0,'6':0}

for game in range(NUM_GAMES):
  answer = random.choice(answers)
  guess = random.choice(guesses)
  print('NEW GAME NO:',game+1)
  print('ANSWER:',answer)
  absent,present,pos_present,pos_absent = play(answer, guess,[],[],list('-----'),['','','','',''])
  print('1 guess:',guess)
  for number_play in list(range(2,7)):
    if number_play < 7:
        
      if list(answer) != list(guess):
        remain_words = eliminate_words(absent, present, pos_present,pos_absent)
        sorted_entropy,sorted_relv_entropy = get_entropy2(remain_words,freqs_dict,COMBS)
        guess = sorted_entropy[0][0]
        print(number_play,'guess:',guess)
        #play
        absent,present,pos_present,pos_absent = play(answer, guess,absent,present,pos_present,pos_absent)
        
        
      if list(answer)==list(guess):
        print('{} guessed in {} guesses'.format(guess,number_play))
        NUM_WINS +=1
        win_dict[str(number_play)] +=1
        break
        

    if number_play==6:

      if list(answer) == list(guess):
        print('Answer gotten at the Last step:',guess)
        NUM_WINS +=1
        win_dict[str(number_play)] +=1

      else:
        print('LOST')
        print('End of the Game & the Final Guess:',guess)
  print()

print('Number of games won out of {} is {}'.format(NUM_GAMES,NUM_WINS))
print('Number of wins for each number of guesses:')
print(win_dict)

NEW GAME NO: 1
ANSWER: worse
1 guess: rubus
2 guess: sawer
3 guess: wersh
4 guess: worse
worse guessed in 4 guesses

NEW GAME NO: 2
ANSWER: bevel
1 guess: guyed
2 guess: pares
3 guess: motel
4 guess: incel
5 guess: bevel
bevel guessed in 5 guesses

NEW GAME NO: 3
ANSWER: chest
1 guess: inust
2 guess: reast
3 guess: ghest
4 guess: chest
chest guessed in 4 guesses

NEW GAME NO: 4
ANSWER: thank
1 guess: urbia
2 guess: pleas
3 guess: tango
4 guess: thank
thank guessed in 4 guesses

NEW GAME NO: 5
ANSWER: prize
1 guess: hecks
2 guess: pared
3 guess: preon
4 guess: prize
prize guessed in 4 guesses

Number of games won out of 5 is 5
Number of wins for each number of guesses:
{'1': 0, '2': 0, '3': 0, '4': 4, '5': 1, '6': 0}


# **END**